In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
import numpy as np
import pandas as pd
import h5py
from tqdm import tqdm
import scipy.io as sio
from scipy import stats
from importlib import reload
from matplotlib import pyplot as plt
import sys
sys.path.append('/home/jma819/post_cmfe_analysis')
#sys.path.append('/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/post_cmfe_analysis')
import os
os.chdir('/home/jma819/post_cnmfe_analysis/')
import python_utils_jjm as utils_jjm
import python_utils_jjm as utils_jjm
import dlc_utils
from sklearn.preprocessing import MinMaxScaler
import scipy.spatial.distance as dist
import itertools
import math
import warnings
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML
import av
from multiprocessing import Pool
import functools
import glob
plt.rcParams['animation.ffmpeg_path']='/home/jma819/.conda/envs/caiman/bin/ffmpeg'
warnings.filterwarnings(action='once')

In [2]:
## list *out.mat files to analyze

/home/jma819/.conda/envs/caiman_with_tables/lib/python3.8/site-packages/jupyter_client/jsonutil.py:64: DeprecationWarning: Interpreting naive datetime as local 2022-11-21 15:57:44.638252. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)
/home/jma819/.conda/envs/caiman_with_tables/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
dir_path = '/projects/p30771/MATLAB/CNMF_E_jjm/quest_MATLAB_cnmfe/batch_output_files/'
file_name = '20-Nov_23_49_28_out.mat'
CNMFE_file = dir_path+file_name

## load cnmfe output and perform some basic adjustments (normalize traces to peak intensity, calculate z score)
cell_fluorescence = sio.loadmat(CNMFE_file)
C_timedelta = utils_jjm.create_fluorescence_time_delta(cell_fluorescence['C'])
C_normalized = C_timedelta.apply(utils_jjm.normalize).set_index(pd.to_timedelta(np.linspace(0, (len(C_timedelta)-1)*(1/20), len(C_timedelta)), unit='s'), drop=True)
C_z_scored = C_timedelta.apply(stats.zscore).set_index(pd.to_timedelta(np.linspace(0, (len(C_timedelta)-1)*(1/20), len(C_timedelta)), unit='s'), drop=True)
C_normalized_z_scored = C_normalized.apply(stats.zscore).set_index(pd.to_timedelta(np.linspace(0, (len(C_normalized)-1)*(1/20), len(C_normalized)), unit='s'), drop=True)

spatial_components=np.array(cell_fluorescence['A'].todense())

##load spatial components by session
# for v4 dimensions are 600x600 pixels
com_df, spatial_components = utils_jjm.return_spatial_info(CNMFE_file, 0.6, dims=(600, 600))
cell_contours, for_dims = utils_jjm.create_contour_layouts(spatial_components, dims=(600, 600))

#cell_fluorescence['P'][0][0][13][0][0][0].split('/')

print(cell_fluorescence['P'][0][0][13][0][0][0].split('/')[5]+'_'+cell_fluorescence['P'][0][0][13][0][0][0].split('/')[6]+'_'+cell_fluorescence['P'][0][0][13][0][0][0].split('/')[8])

grouped_raw_data = {}

session = cell_fluorescence['P'][0][0][13][0][0][0].split('/')[5]+'_'+cell_fluorescence['P'][0][0][13][0][0][0].split('/')[6]+'_'+cell_fluorescence['P'][0][0][13][0][0][0].split('/')[8]
grouped_raw_data[session] = {'C': C_timedelta, 'C_z_scored': C_z_scored, 'C_normalized': C_normalized, 
                               'C_normalized_z_scored': C_normalized_z_scored, 'com' : com_df, 
                               'spatial_components' : spatial_components, 
                               'cell_contours': cell_contours, 'for_dims' : for_dims}

## for spatial clustering start by using z scored data on whole trace as input to event detection function, then try with z score based 
#on rest or other periods
C_traces_filtered_for_analysis = {}
com_filtered_for_analysis = {}
##remove small cells 
for session in list(grouped_raw_data.keys()):
    # get indicies of small cells 
    cells_to_drop = np.array([int(cell) for cell in range(1, len(grouped_raw_data[session]['cell_contours'])+1) if 
                              len(np.array(np.where(grouped_raw_data[session]['for_dims'][cell]>0.6)[0]))<100])
    C_traces_filtered = grouped_raw_data[session]['C_normalized_z_scored'].drop(cells_to_drop, axis=1)
    
    C_traces_filtered_for_analysis[session] = C_traces_filtered.drop('msCamFrame', axis=1)
    com_filtered_for_analysis[session] = grouped_raw_data[session]['com'].drop(cells_to_drop, axis=0).transpose()

print('cell count:')
print(np.shape(C_traces_filtered)[1])

# sets sampling interval to 5Hz to correspond to behavior data 
new_sampling_interval = .2
C_z_score_df = pd.concat([C_traces_filtered_for_analysis[session].resample(str(new_sampling_interval)+'S').max()
                  for session in list(C_traces_filtered_for_analysis.keys())], axis=1, keys=list(C_traces_filtered_for_analysis.keys()))

# after downsampling convert back to regular index, timedelta is behaving oddly 
C_z_score_df_reindexed = pd.concat([C_z_score_df[session].reset_index(drop=True) 
                                    for session in list(set(C_z_score_df.columns.get_level_values(0)))], axis=1, keys=list(set(C_z_score_df.columns.get_level_values(0))))
# drop columns with all nan values
C_z_score_df_reindexed.dropna(axis=1, how='all', inplace=True)

print(session)

## save each session csv file individually for loading into MATLAB
# using MATLAB readtable function, which will create variable names from column header 
save_directory = '/projects/b1118/miniscope/analysis/spatial_data/spatial_clustering/jones_script_analysis/'
for session in list(set(C_z_score_df_reindexed.columns.get_level_values(0))):
    C_z_score_df_reindexed[session].to_csv(save_directory+session+'_C_traces_filtered.csv')
    com_filtered_for_analysis[session].to_csv(save_directory+session+'_com_filtered.csv')

print('finished, saved:')
list(set(C_z_score_df_reindexed.columns.get_level_values(0)))

/home/jma819/.conda/envs/caiman_with_tables/lib/python3.8/site-packages/jupyter_client/jsonutil.py:64: DeprecationWarning: Interpreting naive datetime as local 2022-11-21 16:17:55.627409. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)
/home/jma819/.conda/envs/caiman_with_tables/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Vehicle_DIO_r2.14_17_40_46
cell count:
20
Vehicle_DIO_r2.14_17_40_46
finished, saved:


['Vehicle_DIO_r2.14_17_40_46']

In [68]:
## batch compiling
dir_path = '/projects/p30771/MATLAB/CNMF_E_jjm/quest_MATLAB_cnmfe/batch_output_files/'
file_names = []

/home/jma819/.conda/envs/caiman_with_tables/lib/python3.8/site-packages/jupyter_client/jsonutil.py:64: DeprecationWarning: Interpreting naive datetime as local 2022-11-21 11:59:03.319321. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [ ]:
key = {}
for file_name in file_names:
    CNMFE_file = dir_path+file_name
    ## load cnmfe output and perform some basic adjustments (normalize traces to peak intensity, calculate z score)
    cell_fluorescence = sio.loadmat(CNMFE_file)
    C_timedelta = utils_jjm.create_fluorescence_time_delta(cell_fluorescence['C'])
    C_normalized = C_timedelta.apply(utils_jjm.normalize).set_index(pd.to_timedelta(np.linspace(0, (len(C_timedelta)-1)*(1/20), len(C_timedelta)), unit='s'), drop=True)
    C_z_scored = C_timedelta.apply(stats.zscore).set_index(pd.to_timedelta(np.linspace(0, (len(C_timedelta)-1)*(1/20), len(C_timedelta)), unit='s'), drop=True)
    C_normalized_z_scored = C_normalized.apply(stats.zscore).set_index(pd.to_timedelta(np.linspace(0, (len(C_normalized)-1)*(1/20), len(C_normalized)), unit='s'), drop=True)
    spatial_components=np.array(cell_fluorescence['A'].todense())
    ##load spatial components by session
    # for v4 dimensions are 600x600 pixels
    com_df, spatial_components = utils_jjm.return_spatial_info(CNMFE_file, 0.6, dims=(600, 600))
    cell_contours, for_dims = utils_jjm.create_contour_layouts(spatial_components, dims=(600, 600))
    # cell_fluorescence['P'][0][0][13][0][0][0].split('/')
    print(cell_fluorescence['P'][0][0][13][0][0][0].split('/')[5]+'_'+cell_fluorescence['P'][0][0][13][0][0][0].split('/')[6]+'_'+cell_fluorescence['P'][0][0][13][0][0][0].split('/')[8])
    grouped_raw_data = {}
    session = cell_fluorescence['P'][0][0][13][0][0][0].split('/')[5]+'_'+cell_fluorescence['P'][0][0][13][0][0][0].split('/')[6]+'_'+cell_fluorescence['P'][0][0][13][0][0][0].split('/')[8]
    grouped_raw_data[session] = {'C': C_timedelta, 'C_z_scored': C_z_scored, 'C_normalized': C_normalized, 
                               'C_normalized_z_scored': C_normalized_z_scored, 'com' : com_df, 
                               'spatial_components' : spatial_components, 
                               'cell_contours': cell_contours, 'for_dims' : for_dims}
    ## for spatial clustering start by using z scored data on whole trace as input to event detection function, then try with z score based 
    #on rest or other periods
    C_traces_filtered_for_analysis = {}
    com_filtered_for_analysis = {}
    ##remove small cells 
    for session in list(grouped_raw_data.keys()):
        # get indicies of small cells 
        cells_to_drop = np.array([int(cell) for cell in range(1, len(grouped_raw_data[session]['cell_contours'])+1) if 
                              len(np.array(np.where(grouped_raw_data[session]['for_dims'][cell]>0.6)[0]))<100])
        C_traces_filtered = grouped_raw_data[session]['C_normalized_z_scored'].drop(cells_to_drop, axis=1)
    
        C_traces_filtered_for_analysis[session] = C_traces_filtered.drop('msCamFrame', axis=1)
        com_filtered_for_analysis[session] = grouped_raw_data[session]['com'].drop(cells_to_drop, axis=0).transpose()
    print('cell count:')
    print(np.shape(C_traces_filtered)[1])
    # sets sampling interval to 5Hz to correspond to behavior data 
    new_sampling_interval = .2
    C_z_score_df = pd.concat([C_traces_filtered_for_analysis[session].resample(str(new_sampling_interval)+'S').max()
                  for session in list(C_traces_filtered_for_analysis.keys())], axis=1, keys=list(C_traces_filtered_for_analysis.keys()))
    # after downsampling convert back to regular index, timedelta is behaving oddly 
    C_z_score_df_reindexed = pd.concat([C_z_score_df[session].reset_index(drop=True) 
                                    for session in list(set(C_z_score_df.columns.get_level_values(0)))], axis=1, keys=list(set(C_z_score_df.columns.get_level_values(0))))
    # drop columns with all nan values
    C_z_score_df_reindexed.dropna(axis=1, how='all', inplace=True)
    print(session)
    ## save each session csv file individually for loading into MATLAB
    # using MATLAB readtable function, which will create variable names from column header 
    save_directory = '/projects/b1118/miniscope/analysis/spatial_data/spatial_clustering/jones_script_analysis/'
    for session in list(set(C_z_score_df_reindexed.columns.get_level_values(0))):
        C_z_score_df_reindexed[session].to_csv(save_directory+session+'_C_traces_filtered.csv')
        com_filtered_for_analysis[session].to_csv(save_directory+session+'_com_filtered.csv')
    print('finished, saved:')
    list(set(C_z_score_df_reindexed.columns.get_level_values(0)))
    key[session]=file_name